In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train[['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'SalePrice']]
test=test[['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF']]
train.head(5)

,LotFrontage,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,SalePrice
0,65.0,7,2003,2003,196.0,706,856,856,854,1710,2,8,0,2003.0,2,548,0,61,208500
1,80.0,6,1976,1976,0.0,978,1262,1262,0,1262,2,6,1,1976.0,2,460,298,0,181500
2,68.0,7,2001,2002,162.0,486,920,920,866,1786,2,6,1,2001.0,2,608,0,42,223500
3,60.0,7,1915,1970,0.0,216,756,961,756,1717,1,7,1,1998.0,3,642,0,35,140000
4,84.0,8,2000,2000,350.0,655,1145,1145,1053,2198,2,9,1,2000.0,3,836,192,84,250000


In [2]:
cols = ['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF']

#feat_dict = {}

# for col in cols:
#     training = train[col].values.tolist()
#     testing = test[col].values.tolist()
#     comb = training.extend(testing)
#     feat_dict[col] = set(comb)
    

In [3]:
#print(np.array(train['MSSubClass'].replace(feat_dict['MSSubClass']))[0:10])

In [10]:
def genarr(data, cols):
    X = np.array([], dtype=np.float32)
    Y = np.array(data['SalePrice'], dtype=np.float32)
    for i in cols:
        tmp = np.array(data[i].fillna(-1), dtype=np.float32)
        X = np.concatenate((X, tmp.T), axis = 0)
    return (np.reshape(X,(len(cols),-1))).T, Y
X,Y = genarr(train, cols)

print(X.shape)
print(Y.shape)
print(X[0:5])
print(Y[0:5])
train.head(20)
print(len(set(Y)))
print(len(Y))

(1460, 18)
(1460,)
[[6.500e+01 7.000e+00 2.003e+03 2.003e+03 1.960e+02 7.060e+02 8.560e+02
  8.560e+02 8.540e+02 1.710e+03 2.000e+00 8.000e+00 0.000e+00 2.003e+03
  2.000e+00 5.480e+02 0.000e+00 6.100e+01]
 [8.000e+01 6.000e+00 1.976e+03 1.976e+03 0.000e+00 9.780e+02 1.262e+03
  1.262e+03 0.000e+00 1.262e+03 2.000e+00 6.000e+00 1.000e+00 1.976e+03
  2.000e+00 4.600e+02 2.980e+02 0.000e+00]
 [6.800e+01 7.000e+00 2.001e+03 2.002e+03 1.620e+02 4.860e+02 9.200e+02
  9.200e+02 8.660e+02 1.786e+03 2.000e+00 6.000e+00 1.000e+00 2.001e+03
  2.000e+00 6.080e+02 0.000e+00 4.200e+01]
 [6.000e+01 7.000e+00 1.915e+03 1.970e+03 0.000e+00 2.160e+02 7.560e+02
  9.610e+02 7.560e+02 1.717e+03 1.000e+00 7.000e+00 1.000e+00 1.998e+03
  3.000e+00 6.420e+02 0.000e+00 3.500e+01]
 [8.400e+01 8.000e+00 2.000e+03 2.000e+03 3.500e+02 6.550e+02 1.145e+03
  1.145e+03 1.053e+03 2.198e+03 2.000e+00 9.000e+00 1.000e+00 2.000e+03
  3.000e+00 8.360e+02 1.920e+02 8.400e+01]]
[208500. 181500. 223500. 140000. 250000.]
663

In [63]:
import tqdm
import torch
import torch.nn as nn
from torch import optim

class NET(nn.Module):
    def __init__(self,NUM_IN, NUM_OUT):
        super(NET, self).__init__()
        self.linear = nn.Linear(NUM_IN, 1000)
        self.linear1 = nn.Linear(1000,1000)
        self.linear2 = nn.Linear(1000, NUM_OUT)
        self.relu = nn.Tanh();
        self.logSoftmax = nn.Softmax(dim=0)

    def forward(self, X):
        #TODO: Implement forward computation.
        x = torch.tensor(X)
        x = x.clone().detach().to("cuda:0")
        return torch.squeeze(self.logSoftmax(self.linear2(self.relu(self.linear1(self.relu(self.linear(x)))))))

class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

def Train(net, X, Y, n_iter, saleprice, num_classes):
    print("Start Training!")
    #TODO: initialize optimizer.
    optimizer = optim.Adam(net.parameters(), lr=0.0001)
    for epoch in range(n_iter):
        num_correct = 0
        total_loss = 0.0
        net.train()   #Put the network into training mode
        for i in tqdm.notebook.tqdm(range(len(X))):
            #TODO: compute gradients, do parameter update, compute loss.
            x        = X[i]
            y_onehot = torch.zeros(num_classes, device="cuda:0")
            y_onehot[saleprice.index(Y[i])] = 1
            
            net.zero_grad()
            price = net.forward(x)
            func = RMSLELoss()
            loss = func(price, y_onehot)
            total_loss += loss
            loss.backward()
            optimizer.step()
        
        net.eval()    #Switch to eval mode
        print(f"loss on epoch {epoch} = {total_loss}")

priceList = list(set(Y))

priceList.sort()
        
network = NET(len(cols),663).cuda()
Train(network, X, Y, 5, priceList, 663)

Start Training!


  0%|          | 0/1460 [00:00<?, ?it/s]

loss on epoch 0 = 39.29405975341797


  0%|          | 0/1460 [00:00<?, ?it/s]

loss on epoch 1 = 39.270450592041016


...

loss on epoch 2 = 39.24626922607422


  0%|          | 0/1460 [00:00<?, ?it/s]

loss on epoch 3 = 39.19734191894531


  0%|          | 0/1460 [00:00<?, ?it/s]

loss on epoch 4 = 39.16597366333008


In [64]:
for i in range(20):
    print(str(priceList[int(torch.argmax(network.forward(X[i])))])+" "+str(Y[i]))


112000.0 208500.0
144000.0 181500.0
112000.0 223500.0
112000.0 140000.0
310000.0 250000.0
112000.0 143000.0
394617.0 307000.0
310000.0 200000.0
112000.0 129900.0
112000.0 118000.0
120500.0 129500.0
310000.0 345000.0
120500.0 144000.0
287090.0 279500.0
112000.0 157000.0
112000.0 132000.0
120500.0 149000.0
90000.0 90000.0
120500.0 159000.0
112000.0 139000.0
